In [296]:
import pandas as pd
import numpy as np 
pd.options.display.max_colwidth = 100

### RESTAURANTES UBER EATS COLOMBIA. SCRAPPEADOS: 9 JUNIO 2020 3-4 PM.

### Se carga la lista de restaurantes scrappeados:

In [297]:
restaurantes=pd.read_csv("restaurantes_09062020pm.csv")

In [298]:
print("Se tienen: ", len(restaurantes), " restaurantes")

Se tienen:  2535  restaurantes


In [299]:
restaurantes.head()

,Unnamed: 0,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url
0,0,Piali,$ • Pizza,4.5,(33),"Carrera 22 #9-45, Armenia, Armenia 630003 Colombia, Armenia 110221 • Más información",https://www.ubereats.com/co/armenia/food-delivery/piali/gcDAW8dkSpGFa7G-nvkehw
1,1,Urban Pizzería (Armenia),$$ • Pizza,4.6,(91),"Av. Centenario Carrera 6, Cra 6 #8N-40, Armenia, Quindío 630004 • Más información",https://www.ubereats.com/co/armenia/food-delivery/urban-pizzeria-armenia/bVpxAzDDQ42Mv9IAvmkvtg
2,2,Mundial Pizza's,$$ • Pizza,NaN,NaN,"Av Bolívar # 19-46, Armenia, Quindio 110221 • Más información",https://www.ubereats.com/co/armenia/food-delivery/mundial-pizzas/LtB5pxmyRNWVPjF85K4qgQ
3,3,Oeste Pizza,$$ • Italiana • Pizza,4.5,(11),"Cra. 14a, Armenia, Quindío, Colombia, • Más información",https://www.ubereats.com/co/armenia/food-delivery/oeste-pizza/1hb90vymQFyQ_m_y8zMZ8Q
4,4,Parma Pizza,$$ • Italiana,4.4,(31),"Cra. 6 ##3180, Armenia, Quindío, Colombia, Local 220, • Más información",https://www.ubereats.com/co/armenia/food-delivery/parma-pizza/umiGVtjiS2uz-8QYSa3OVw


### Existen resturantes que figuran en varias categorías generales, y tendrán registros duplicados, se eliminan:

In [300]:
restaurantes.drop("Unnamed: 0" ,axis=1, inplace=True) ##Borrar columna innecesaria que proviene de la extracción.
restaurantes.drop_duplicates(keep="first",inplace=True) ## Borrar duplicados

### Se obtienen finalmente 1073 restaurantes diferentes

In [301]:
restaurantes.shape ##Tamaño del dataset

(1073, 6)

### Variables:

In [302]:
restaurantes.columns ##Columnas

Index(['Nombre', 'Categoria_Especifica', 'Calificacion', 'Reviews',
       'Direccion', 'Url'],
      dtype='object')

### * NOMBRE: 
Nombre con el cuál se identifica el restaurante en la página. (String)
### * CATEGORIA_ ESPECIFICA:
 Diferentes categorías en las cuales se puede ubicar el restaurante. Puede ser multivaluado. Es necesario hacer una limpieza, ya que los datos vienen sucios. (string)

### * CALIFICACIÓN: 
Puntuación que reporta la página para el restaurante. (float)

### * REVIEWS: 
Número de reviews que presenta la página para el restaurante. Es necesario hacer limpieza, ya que viene con caracteres sucios. (String)

### * Dirección:
 Ubicación que reporta la página para el restaurante. Es necesario hacer limpieza, ya que viene con datos sucios.

### * URL: 
URL exacta con la cuál se identifica el restaurante en la página.


### Se observa el dtypes:

In [303]:
print("Tipos de datos: ") 
restaurantes.dtypes ##Tipos de las variables

Tipos de datos: 


Nombre                   object
Categoria_Especifica     object
Calificacion            float64
Reviews                  object
Direccion                object
Url                      object
dtype: object

### Buscar valores faltantes: 

In [304]:
print("Datos faltantes por cada variable: ")
restaurantes.isna().sum() ##Mostrar faltantes por variable

Datos faltantes por cada variable: 


Nombre                    0
Categoria_Especifica      1
Calificacion            564
Reviews                 564
Direccion                 2
Url                       0
dtype: int64

### CALIFICACIÓN Y REVIEWS FALTANTES:
 La mayor cantidad de los faltantes en los registros, está en calificación y número de reviews. Esto es porque la mayor parte de los restaurantes cuando están cerrados ocultan su calificación y número de reviews. Adicional a eso, cuando un restaurante es nuevo en la página, tampoco reporta calificación y reviews. Finalmente, dada la situación actual (covid), muchos restaurantes no están operando y salen no disponibles en la plataforma; por ende, ocultan su información de calificación y reviews.

### RESTO DE VARIABLES FALTANTES:
El resto de variables faltantes en los registros, que son muy pocas, es porque el restaurante no tenía la información disponible.

### LIMPIEZA DE DATOS:

#### * DIRECCIÓN:

In [305]:
restaurantes["Direccion"].head()

0    Carrera 22 #9-45, Armenia, Armenia 630003 Colombia, Armenia 110221 • Más información
1       Av. Centenario Carrera 6, Cra 6 #8N-40, Armenia, Quindío 630004 • Más información
2                           Av Bolívar # 19-46, Armenia, Quindio 110221 • Más información
3                                Cra. 14a, Armenia, Quindío, Colombia,  • Más información
4                Cra. 6 ##3180, Armenia, Quindío, Colombia, Local 220,  • Más información
Name: Direccion, dtype: object

*Se observa que tienen el string " • Mas información" en todos lo registros. Se procede a eliminar. Así mismo se borran espacios en blanco al comienzo y final.


In [306]:
restaurantes["Direccion"]=restaurantes["Direccion"].apply(lambda x : str(x).replace("•", "").replace("Más información","").strip()) ##Borrar string y espacios no deseados

In [307]:
restaurantes["Direccion"][1] ##Prueba

'Av. Centenario Carrera 6, Cra 6 #8N-40, Armenia, Quindío 630004'

### * REVIEWS:

In [308]:
restaurantes["Reviews"].value_counts() ##Tipos de valores en reviews

(100+)    28
(200+)    27
(13)      21
(17)      18
(12)      18
          ..
(172)      1
(179)      1
(173)      1
(93)       1
(62)       1
Name: Reviews, Length: 119, dtype: int64

Se nota que hay paréntesis y signos "+" no deseados, pues el interes está únicamente en el valor entero. Se procede a eliminar el signo "+" y los parentesis. Adicional a eso, se hace la conversión a columna numérica.

In [309]:
restaurantes["Reviews"]=restaurantes["Reviews"].apply(lambda x : str(x).replace("+", "").replace("(","").replace(")","").strip()) ##Borrar string y espacios

In [310]:
restaurantes['Reviews'] = restaurantes.Reviews.astype('float').astype("Int64") ##Convertir a tipo entero
print(restaurantes["Reviews"].dtype)
restaurantes["Reviews"].head()


Int64


0      33
1      91
2    <NA>
3      11
4      31
Name: Reviews, dtype: Int64

### CATEGORÍA ESPECIFICA:

*Puede verse que todas los registros tienen un " \$ " o " \$\$ ".
 No sé sabe con seguridad lo que significa esto, pero parece importante almacenar si tiene uno o dos. Se retirarán dichos signos del registro de categoría, pero se creará otra variable que diga cuantos "$" tenía.

 *Adicional a esto, también se ve la presencia del carater " • "

In [311]:
restaurantes["Categoria_Especifica"].head()

0                $ • Pizza
1               $$ • Pizza
2               $$ • Pizza
3    $$ • Italiana • Pizza
4            $$ • Italiana
Name: Categoria_Especifica, dtype: object

In [312]:
restaurantes['Cantidad_$']=restaurantes["Categoria_Especifica"].apply(lambda x: str(x).count("$")) ## Obtener conteo de $
restaurantes["Categoria_Especifica"]=restaurantes["Categoria_Especifica"].apply(lambda x: str(x).replace("$","")) ##Borrar $

Quedó correctamente configurada la nueva variable:

In [313]:
restaurantes["Cantidad_$"].head()

0    1
1    2
2    2
3    2
4    2
Name: Cantidad_$, dtype: int64

Ahora se procede a tratar con el caracter "•" , parece indicar que es el encargado de separar cuando un restaurante pertenece a múltiples categorías. Se evidencia en la siguiente celda:

* Se observan registros como: "• Asiática • Japonesa • Tailandesa • Sushi • Saludable" , donde el carácter "• " separa categorías. Es importante entonces, antes de borrar dicho carácter, lograr atrapar las diferentes categorías que posee el restaurante.

In [314]:
restaurantes["Categoria_Especifica"][44:50] ##Mostrar evidencia

125                                     • Sándwich • Saludable
130                                     • Sándwich • Saludable
141                                    • Helado y yogur helado
148     • Asiática • Japonesa • Tailandesa • Sushi • Saludable
149                                                 • Asiática
151     • Asiática • Japonesa • Tailandesa • Sushi • Saludable
Name: Categoria_Especifica, dtype: object

* Haciendo uso del " • ", se puede hacer un split de cada string con base en dicho carácter, para así guardar las categorías como una lista. Adicional a eso, se puede crear otra variable que indique el número de categorías a las que pertenece cada restaurante: 

In [315]:
restaurantes["Categoria_Especifica"]=restaurantes["Categoria_Especifica"].apply(lambda x: str(x).strip().split("•")[1:]) ## Hacer un split con base en el carácter, obtener desde la segunda hasta la última posición (la primera es vacío)
restaurantes["Numero_Categorias"]=restaurantes["Categoria_Especifica"].apply(lambda x: len(x)) ##Obtener el tamaño de cada lista
restaurantes.head()

,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url,Cantidad_$,Numero_Categorias
0,Piali,[ Pizza],4.5,33,"Carrera 22 #9-45, Armenia, Armenia 630003 Colombia, Armenia 110221",https://www.ubereats.com/co/armenia/food-delivery/piali/gcDAW8dkSpGFa7G-nvkehw,1,1
1,Urban Pizzería (Armenia),[ Pizza],4.6,91,"Av. Centenario Carrera 6, Cra 6 #8N-40, Armenia, Quindío 630004",https://www.ubereats.com/co/armenia/food-delivery/urban-pizzeria-armenia/bVpxAzDDQ42Mv9IAvmkvtg,2,1
2,Mundial Pizza's,[ Pizza],NaN,<NA>,"Av Bolívar # 19-46, Armenia, Quindio 110221",https://www.ubereats.com/co/armenia/food-delivery/mundial-pizzas/LtB5pxmyRNWVPjF85K4qgQ,2,1
3,Oeste Pizza,"[ Italiana , Pizza]",4.5,11,"Cra. 14a, Armenia, Quindío, Colombia,",https://www.ubereats.com/co/armenia/food-delivery/oeste-pizza/1hb90vymQFyQ_m_y8zMZ8Q,2,2
4,Parma Pizza,[ Italiana],4.4,31,"Cra. 6 ##3180, Armenia, Quindío, Colombia, Local 220,",https://www.ubereats.com/co/armenia/food-delivery/parma-pizza/umiGVtjiS2uz-8QYSa3OVw,2,1


* URL:
La URL no necesita ninguna corrección, solo se usará para obtener el nombre de la ciudad donde está el restaurante. Dicho nombre de la ciudad está explicito en la URL. Se almacenará dicha ciudad en una nueva variable ciudad. Para lograr esto se hará un juego con el string de la URL, conocida ya su estructura:

In [316]:
restaurantes["Ciudad"]=restaurantes["Url"].apply(lambda a: a[a.index("/co/"):a.index("/food-")].split("/")[2].capitalize() )
##Obtener la ciudad, jugando con los indices de la estructura de texto conocida
restaurantes.head(3)


,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url,Cantidad_$,Numero_Categorias,Ciudad
0,Piali,[ Pizza],4.5,33,"Carrera 22 #9-45, Armenia, Armenia 630003 Colombia, Armenia 110221",https://www.ubereats.com/co/armenia/food-delivery/piali/gcDAW8dkSpGFa7G-nvkehw,1,1,Armenia
1,Urban Pizzería (Armenia),[ Pizza],4.6,91,"Av. Centenario Carrera 6, Cra 6 #8N-40, Armenia, Quindío 630004",https://www.ubereats.com/co/armenia/food-delivery/urban-pizzeria-armenia/bVpxAzDDQ42Mv9IAvmkvtg,2,1,Armenia
2,Mundial Pizza's,[ Pizza],NaN,<NA>,"Av Bolívar # 19-46, Armenia, Quindio 110221",https://www.ubereats.com/co/armenia/food-delivery/mundial-pizzas/LtB5pxmyRNWVPjF85K4qgQ,2,1,Armenia


In [317]:
restaurantes=restaurantes[["Nombre","Ciudad","Categoria_Especifica","Numero_Categorias","Cantidad_$","Calificacion","Reviews","Direccion","Url"]]

In [318]:
restaurantes.reset_index(inplace=True) ##Organizar indices del dataset
restaurantes.drop("index",axis=1,inplace=True) #borrar indice viejo
restaurantes.head(3) ##Muestradefinitiva. 



,Nombre,Ciudad,Categoria_Especifica,Numero_Categorias,Cantidad_$,Calificacion,Reviews,Direccion,Url
0,Piali,Armenia,[ Pizza],1,1,4.5,33,"Carrera 22 #9-45, Armenia, Armenia 630003 Colombia, Armenia 110221",https://www.ubereats.com/co/armenia/food-delivery/piali/gcDAW8dkSpGFa7G-nvkehw
1,Urban Pizzería (Armenia),Armenia,[ Pizza],1,2,4.6,91,"Av. Centenario Carrera 6, Cra 6 #8N-40, Armenia, Quindío 630004",https://www.ubereats.com/co/armenia/food-delivery/urban-pizzeria-armenia/bVpxAzDDQ42Mv9IAvmkvtg
2,Mundial Pizza's,Armenia,[ Pizza],1,2,NaN,<NA>,"Av Bolívar # 19-46, Armenia, Quindio 110221",https://www.ubereats.com/co/armenia/food-delivery/mundial-pizzas/LtB5pxmyRNWVPjF85K4qgQ


In [322]:
restaurantes.to_excel('Colombia_Limpios.xlsx', encoding='utf-8')